In [1]:
import ee
import geemap
import os
import geopandas as gpd
ee.Initialize()

home = "/Volumes/GoogleDrive/My Drive/Chapter2_mechanisms_forest_water_cycling"

In [11]:
# Find pixels that were forested from 1984 - 2021 using NLCD data at 30m resolution
roi = ee.FeatureCollection('users/kmcquil/blue_ridge_plus_reference_xl').geometry()

# Import the NLCD collection.
dataset = ee.ImageCollection('USGS/NLCD_RELEASES/2016_REL')

# forest is 41, 42, 43 
def LC(y):
    nlcd = dataset.filter(ee.Filter.eq('system:index', y))
    landcover = nlcd.select('landcover').mosaic().clip(roi)
    return landcover.eq(41).max(landcover.eq(42)).max(landcover.eq(43))

# apply mask to all years 
years = ee.List(['1992', '2001', '2004', '2006', '2008', '2011', '2013', '2016'])
lc_mask_years = ee.ImageCollection.fromImages(years.map(LC).flatten())

# sum all of the years 
lc_sum = lc_mask_years.sum()

# there are 8 years, so we only want pixels = 8 
lc_sum8 = lc_sum.eq(8)
lc_sum8 = lc_sum8.updateMask(lc_sum8)

# export the permanent forest cover layer 
task_config = {
        'region': roi,
        'fileFormat': 'GeoTIFF',
        'folder':'nlcd_permanent_forest',
        'description': 'landsat_permanent_forest',
        'image': lc_sum8,
        'maxPixels':1e13, 
        'scale' : 30
    }

task=ee.batch.Export.image.toDrive(**task_config)
task.start()

task2 = ee.batch.Export.image.toAsset(**{
  'image': lc_sum,
  'description': 'sum of forest pixels',
  'assetId': 'users/kmcquil/nlcd_sum_forest',
  'scale': 30,
  'region':roi, #roi.getInfo()['coordinates']
  'maxPixels':1e13
})
task2.start()

In [5]:
# convert the gpkg to a shapefile and upload to GEE (use code editor asset uploader )
hws = gpd.read_file(os.path.join(home, "Data", "Catchments", "Headwater", "headwater_catchments_sbr.gpkg"))
hws4326 = hws.to_crs("EPSG:4326")
hws4326.to_file(os.path.join(home, "Data", "Catchments", "Headwater", "headwater_catchments_sbr_shp.shp"))

In [16]:
# Grab the nlcd for 2001 and 2016 so that I can compare the deciduous vs evergreen percentages by headwater catchment
roi = ee.FeatureCollection('users/kmcquil/blue_ridge_plus_reference_xl').geometry()

# Import the NLCD collection and grab the data for 2001 and 2016 
dataset = ee.ImageCollection('USGS/NLCD_RELEASES/2016_REL')
nlcd_2001 = dataset.filter(ee.Filter.eq('system:index', '2001'))
nlcd_2001 = nlcd_2001.select('landcover').mosaic().clip(roi)

nlcd_2016 = dataset.filter(ee.Filter.eq('system:index', '2016'))
nlcd_2016 = nlcd_2016.select('landcover').mosaic().clip(roi)

# export nlcd 2001 
task_config_2001 = {
        'region': roi,
        'fileFormat': 'GeoTIFF',
        'folder':'nlcd_permanent_forest',
        'description': 'nlcd_2001',
        'image': nlcd_2001,
        'maxPixels':1e13, 
        'scale' : 30
    }

task2001=ee.batch.Export.image.toDrive(**task_config_2001)
task2001.start()

# export nlcd 2016
task_config_2016 = {
        'region': roi,
        'fileFormat': 'GeoTIFF',
        'folder':'nlcd_permanent_forest',
        'description': 'nlcd_2016',
        'image': nlcd_2016,
        'maxPixels':1e13, 
        'scale' : 30
    }

task2016=ee.batch.Export.image.toDrive(**task_config_2016)
task2016.start()